###  Desicion tree to json

**Как строится дерево решений; Энтропия; Критерии качества разбиения**:
- https://habr.com/ru/companies/ods/articles/322534/ 

**Note**:
- tree_.value = [20, 34]: 20 элементов класса 0 и 34 - класса 1

In [45]:
import json
from sklearn.tree import DecisionTreeClassifier


def _convert_tree(tree: DecisionTreeClassifier, node_index: int) -> dict:
    
    is_leaf = tree.children_left[node_index] == -1 and tree.children_right[node_index] == -1
    
    if is_leaf:
        return {"class": int(tree.value[node_index].argmax())}
    else:
        feature_index = int(tree.feature[node_index])
        threshold = float(tree.threshold[node_index])
        left_child_index = tree.children_left[node_index]
        right_child_index = tree.children_right[node_index]
        return {
                "feature_index": feature_index,
                "threshold": round(threshold,4),
                "left":  _convert_tree(tree, left_child_index),
                "right":  _convert_tree(tree, right_child_index)
        }

def convert_tree_to_json(tree: DecisionTreeClassifier) -> str:
    node_index = 0
    dct_tree = _convert_tree(tree.tree_, node_index)
    tree_as_json = json.dumps(dct_tree)
    return tree_as_json

In [46]:
#Создаем синтетический датасет
from sklearn.datasets import make_classification

X, y = make_classification(
    n_samples=100, # observations 
    n_features=5, # 5 total features
    n_informative=3, # 3 'useful' features
    n_classes=2, # binary target/label 
    random_state=999 # if you want the same results
)

In [47]:
from sklearn.tree import DecisionTreeClassifier

# Train decision tree
model = DecisionTreeClassifier()
model.fit(X, y)

DecisionTreeClassifier()

In [48]:
convert_tree_to_json(model)

'{"feature_index": 0, "threshold": 0.1395, "left": {"feature_index": 3, "threshold": -0.6881, "left": {"feature_index": 2, "threshold": 0.3686, "left": {"class": 1}, "right": {"class": 0}}, "right": {"feature_index": 3, "threshold": 0.3453, "left": {"feature_index": 2, "threshold": 0.5135, "left": {"feature_index": 1, "threshold": 0.2244, "left": {"class": 1}, "right": {"class": 0}}, "right": {"feature_index": 1, "threshold": 0.7156, "left": {"class": 0}, "right": {"class": 1}}}, "right": {"class": 1}}}, "right": {"feature_index": 3, "threshold": -0.4706, "left": {"feature_index": 3, "threshold": -2.534, "left": {"class": 0}, "right": {"feature_index": 3, "threshold": -1.0829, "left": {"class": 1}, "right": {"feature_index": 4, "threshold": 1.4327, "left": {"class": 0}, "right": {"feature_index": 1, "threshold": -5.0584, "left": {"class": 0}, "right": {"class": 1}}}}}, "right": {"class": 0}}}'